# Cannibal(ization)! The Notebook

In this notebook, we look at how to do a simple cannibalization study using Overture Maps POI data and Kontur population data. We will also be using the Valhalla open source routing engine.

## Loading Data

We start by importing some libraries that will be useful to us and creating a Sedona context.

In [15]:
import geopandas as gpd
import pandas as pd
import requests
from sedona.spark import *
from pyspark.sql.functions import explode, array
from pyspark.sql import functions as F

In [ ]:
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

We define a function `get_isochrone` which takes as its input the coordinates of a location, the costing function (meaning whether we want to plot isochrones for driving, walking, or biking), what time steps we want isochrones for, and optional arguments for a name and brand ID. 

The function returns a GeoPandas dataframe which has a column for the isochrone geometry, and columns for the point name and brand ID.

In [2]:
def get_isochrone(lat, lng, costing, time_steps, name = 'taargus', store_id = 'taargus'):
    url = "https://valhalla1.openstreetmap.de/isochrone"
    params = {
      "locations": [{"lon": lng, "lat": lat}],
      "contours": [{"time": i} for i in time_steps],
      "costing": costing,
      "polygons": 1,
    }
    response = requests.post(url, json=params)
    if response:
        result = response.json()
        if 'error_code' not in result.keys():
            df = gpd.GeoDataFrame.from_features(result)
            df['name'] = name
            df['id'] = store_id
            return df[['name','id','geometry']]

Next, we load the two main data sources we will need. First, we create a temporary view of the `places` table from Overture Maps. Then, we load the Kontur population dataset.

The Kontur dataset contains estimates for population in the US, aggregated by aperture 8 H3 hexes (~400m side-length). Viewing population by H3 hexes rather than administrative boundaries allows us to estimate populations inside drivetimes or other irregular polygons.

We convert the hex names in the Kontur table to `BIGINT` because this is how H3 hexes are labeled in SedonaSQL. 

In [2]:
sedona.table("wherobots_open_data.overture.places_place").createOrReplaceTempView("places")

kontur = sedona.read.option('header','true').load('s3://wbts-wbc-m97rcg45xi/ouz7h0o9bi/data/shared/h3_population/us_h3_8_pop.geojson', format="json") .\
drop('_corrupt_record').dropna() .\
selectExpr('CAST(CONV(properties.h3, 16, 10) AS BIGINT) AS h3', 'properties.population as population')
kontur.createOrReplaceTempView('kontur')

We will be doing a cannibalization of two primarily Texas-based taco chains: Torchy's Tacos and Velvet Taco.

The `stores` dataframe below selects the Texas locations of both chains. We also add a `chain` column to our dataframe to make it easier to aggregate these locations by chain name later on and not worry about locations with nonstandard names like 'Torchy's Tacos DFW' or 'Velvet Taco Midtown' which may occur.

In [3]:
stores = sedona.sql("""
SELECT id, names.common[0].value as name, ST_X(geometry) as long, ST_Y(geometry) as lat, geometry, CASE WHEN names.common[0].value LIKE "%Torchy's Tacos%" THEN "Torchy's Tacos" ELSE 'Velvet Taco' END AS chain
FROM places
WHERE addresses[0].region = 'TX'
AND (names.common[0].value LIKE "%Torchy's Tacos%" OR names.common[0].value LIKE '%Velvet Taco%')
""")

In [9]:
stores.show()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+--------------------+--------------+-----------+----------+
|                  id|          name|       long|       lat|
+--------------------+--------------+-----------+----------+
|tmp_8104A79216254...|Torchy's Tacos|  -98.59689|  29.60891|
|tmp_D17CA8BD72325...|Torchy's Tacos|  -97.74175|  30.29368|
|tmp_F497329382C10...|   Velvet Taco|  -95.48866|  30.18314|
|tmp_9B40A1BF3237E...|Torchy's Tacos| -96.805853| 32.909982|
|tmp_38210E5EC047B...|Torchy's Tacos|  -96.68755|  33.10118|
|tmp_DF0C5DF6CA549...|Torchy's Tacos|  -97.75159|  30.24542|
|tmp_BE38CAC8D46CF...|Torchy's Tacos|  -97.80877|  30.52676|
|tmp_44390C4117BEA...|Torchy's Tacos|  -97.82594|   30.4547|
|tmp_8032605AA5BDC...|   Velvet Taco| -96.469695| 32.898634|
|tmp_0A2AA67757F42...|Torchy's Tacos|  -96.44858|  32.90856|
|tmp_643821EB9C104...|Torchy's Tacos|  -97.11933|  32.94021|
|tmp_0042962D27E06...|   Velvet Taco|-95.3905374|29.7444214|
|tmp_8D0E2246C3F36...|Torchy's Tacos|  -97.15952|  33.22987|
|tmp_CB939610BC175...|To

In [10]:
stores.count()

124

## Isochrone Generation

This is an optional part of the notebook which lets us create a new isochrone dataframe and save it to our storage.

In [52]:
drivetimes_5_min = pd.concat([get_isochrone(row.lat, row.long, 'auto', [5], row.chain, row.id) for row in stores.select('id','chain','lat','long').collect()])

In [12]:
drivetimes_20_min.shape

(124, 3)

In [60]:
drivetimes_5_min.to_csv('s3://wbts-wbc-m97rcg45xi/ouz7h0o9bi/data/customer-4wwwla6ldgla6q/drivetimes_5_min_torchys_velvet.csv', index = False)

## Cannibalization Analysis

We load an existing isochrone dataframe. For simplicity's sake, we will look at 5-minute drive times for each Torchy's and Velvet Taco location.

In [4]:
sedona.read.option('header','true').format('csv') .\
load('s3://wbts-wbc-m97rcg45xi/ouz7h0o9bi/data/customer-4wwwla6ldgla6q/drivetimes_5_min_torchys_velvet.csv') .\
createOrReplaceTempView('drivetimes_5_min')

We can turn the highly irregular isochrone polygons into collections of H3 hexes. 

We apply the `explode` function to the `h3` column so that we get a table where each row contains information about a single H3 hex rather than a list of hexes, because this will make it easier to compute the drivetime population estimates and plot our results later on.

In [5]:
sedona.sql("""
SELECT ST_H3CellIds(ST_GeomFromWKT(geometry), 8, false) AS h3, name, id
FROM drivetimes_5_min
""").select(explode('h3'), 'name','id').withColumnRenamed('col','h3') .\
join(kontur, 'h3', 'left').distinct().createOrReplaceTempView('h3_isochrones')

We can run the following query to show the number of people in Texas who live within a 5 minutes drive of a Torchy's Tacos:

In [65]:
sedona.sql("""
WITH distinct_h3 (h3, population) AS 
(
    SELECT DISTINCT h3, ANY_VALUE(population)
    FROM h3_isochrones
    WHERE name LIKE "%Torchy's%"
    GROUP BY h3
)
SELECT SUM(population)
FROM distinct_h3
""").show()

+---------------+
|sum(population)|
+---------------+
|      1546765.0|
+---------------+



We can also see how many people live within a 5-minute drivetime of more than one Torchy's Tacos:

In [18]:
sedona.sql("""
SELECT ST_H3CellIds(ST_GeomFromWKT(geometry), 8, false) AS h3, name, id
FROM drivetimes_5_min
""").select(explode('h3'), 'name','id').withColumnRenamed('col','h3') .\
join(kontur, 'h3', 'left').filter('name LIKE "%Torchy%"').select('h3','population') .\
groupBy('h3').count().filter('count >= 2').join(kontur, 'h3', 'left').distinct() .\
agg(F.sum('population')).collect()[0][0]

97903.0

Population in TX living within 5 minutes' of a Velvet Taco:

In [66]:
sedona.sql("""
WITH distinct_h3 (h3, population) AS 
(
    SELECT DISTINCT h3, ANY_VALUE(population)
    FROM h3_isochrones
    WHERE name LIKE '%Velvet Taco%'
    GROUP BY h3
)
SELECT SUM(population)
FROM distinct_h3
""").show()

+---------------+
|sum(population)|
+---------------+
|       750360.0|
+---------------+



We can also see how many people live within a 5-minute drivetime of more than one Velvet Taco:

In [19]:
sedona.sql("""
SELECT ST_H3CellIds(ST_GeomFromWKT(geometry), 8, false) AS h3, name, id
FROM drivetimes_5_min
""").select(explode('h3'), 'name','id').withColumnRenamed('col','h3') .\
join(kontur, 'h3', 'left').filter('name LIKE "%Velvet%"').select('h3','population') .\
groupBy('h3').count().filter('count >= 2').join(kontur, 'h3', 'left').distinct() .\
agg(F.sum('population')).collect()[0][0]

100298.0

Population living in drivetime overlap:

In [67]:
sedona.sql("""
WITH overlap_h3 (h3, population) AS
(
    SELECT DISTINCT a.h3, ANY_VALUE(a.population)
    FROM h3_isochrones a LEFT JOIN h3_isochrones b ON a.h3 = b.h3
    WHERE a.name != b.name
    GROUP BY a.h3
)
SELECT sum(population)
FROM overlap_h3
""").show()

+---------------+
|sum(population)|
+---------------+
|       415033.0|
+---------------+



We see that over half of the people who live within a 5 minute drive of a Velvet Taco also live within a 5 minute drive of a Torchy's Tacos.

## Visualizations

In this section, we show how to use SedonaKepler to plot the locations of the two taco chains we examined, as well as how to visualize the respective drive times and population estimates.

The first map shows the Texas location of each chain, color-coded by the chain id.

In [6]:
location_map_cfg = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'y2oi43e',
     'type': 'geojson',
     'config': {'dataId': 'Locations',
      'label': 'Locations',
      'color': [183, 136, 94],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Ice And Fire 3',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#0198BD', '#FAFEB3', '#D50255']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': False,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'chain', 'type': 'string'},
      'colorScale': 'ordinal',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'Locations': [{'name': 'chain',
        'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 30.200120881683112,
   'longitude': -97.57506424815895,
   'pitch': 0,
   'zoom': 8.797279073806099,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [7]:
location_map = SedonaKepler.create_map(stores, "Locations", config = location_map_cfg)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [8]:
location_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'y2oi43e', 'type': …

In [91]:
location_map.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'y2oi43e',
     'type': 'geojson',
     'config': {'dataId': 'Locations',
      'label': 'Locations',
      'color': [183, 136, 94],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Ice And Fire 3',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#0198BD', '#FAFEB3', '#D50255']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enable

In [6]:
isochrone_map_cfg = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'e4swxqa',
     'type': 'geojson',
     'config': {'dataId': 'Isochrones',
      'label': 'Isochrones',
      'color': [241, 92, 23],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.3,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [34, 63, 154],
       'colorRange': {'name': 'Ice And Fire 3',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#0198BD', '#FAFEB3', '#D50255']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': False,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'name', 'type': 'string'},
      'colorScale': 'ordinal',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'Isochrones': [{'name': 'name',
        'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 32.601205964888095,
   'longitude': -96.40454260677171,
   'pitch': 0,
   'zoom': 8.7972790738061,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [7]:
map_isochrones = sedona.read.option('header','true').format('csv').load('s3://wbts-wbc-m97rcg45xi/ouz7h0o9bi/data/customer-4wwwla6ldgla6q/drivetimes_5_min_torchys_velvet.csv')
isochrone_map = SedonaKepler.create_map(map_isochrones, "Isochrones", config = isochrone_map_cfg)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [8]:
isochrone_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'e4swxqa', 'type': …

In [99]:
isochrone_map.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'e4swxqa',
     'type': 'geojson',
     'config': {'dataId': 'Isochrones',
      'label': 'Isochrones',
      'color': [241, 92, 23],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.3,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [34, 63, 154],
       'colorRange': {'name': 'Ice And Fire 3',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#0198BD', '#FAFEB3', '#D50255']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
    

In [12]:
isochrones_h3_map_cfg = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'ca6k4k',
     'type': 'geojson',
     'config': {'dataId': 'Isochrones in H3',
      'label': 'Isochrones in H3',
      'color': [241, 92, 23],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.3,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [34, 63, 154],
       'colorRange': {'name': 'Ice And Fire 3',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#0198BD', '#FAFEB3', '#D50255']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': False,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'name', 'type': 'string'},
      'colorScale': 'ordinal',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'Isochrones in H3': [{'name': 'name',
        'format': None},
       {'name': 'population', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 30.109638748624587,
   'longitude': -93.94837575861246,
   'pitch': 0,
   'zoom': 5.798979652677288,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}



In [13]:
isochrones_h3_map_data = sedona.sql("""
SELECT ST_H3CellIds(ST_GeomFromWKT(geometry), 8, false) AS h3, name, id
FROM drivetimes_5_min
""").select(explode('h3'), 'name','id').withColumnRenamed('col','h3') .\
join(kontur, 'h3', 'left').select('name','population',array('h3')).withColumnRenamed('array(h3)','h3').selectExpr('name','population','ST_H3ToGeom(h3) AS geometry')

isochrones_h3_map = SedonaKepler.create_map(isochrones_h3_map_data, 'Isochrones in H3', config = isochrones_h3_map_cfg)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/opt/conda/lib/python3.10/site-packages/jupyter_client/session.py:719: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [14]:
isochrones_h3_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ca6k4k', 'type': '…

In [103]:
isochrones_h3_map.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'ca6k4k',
     'type': 'geojson',
     'config': {'dataId': 'Isochrones in H3',
      'label': 'Isochrones in H3',
      'color': [241, 92, 23],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.3,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [34, 63, 154],
       'colorRange': {'name': 'Ice And Fire 3',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#0198BD', '#FAFEB3', '#D50255']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScal